
# TP CNN
### Diane LINGRAND 

diane.lingrand@univ-cotedazur.fr 

# Introduction

In [1]:
from IPython.display import Image
import tensorflow
print(tensorflow.__version__)
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Activation 
import matplotlib.pyplot as plt

2.4.0


In [2]:
tensorflow.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

**The GPU**



To enable GPU backend in Google colab for your notebook:

1.   Runtime (top left corner) -> Change runtime type
2.   Put GPU as "Hardware accelerator"
3.   Save.

# Convolutional Neural Networks (CNN)

Derived from the MLP, a convolutional neural network (CNN) is a type of artificial neural network that is specifically designed to process **pixel data**.  The layers of a CNN consist of an **input layer**, an **output layer** and **hidden layers** that can include **convolutional layers**, **pooling layers**, **fully connected layers** and **normalization layers**. It exists a lot of techniques to optimize CNN, like for example the dropout.

## Loading the dataset
In this part, we will use photographies of animals from the kaggle dataset [animals-10](https://www.kaggle.com/alessiocorrado99/animals10). Please connect to their site before loading the dataset from this [zip file](http://www.i3s.unice.fr/~lingrand/raw-img.zip). Decompress the zip file on your disk

To feed the data to a CNN, we need to shape it as required by Keras. As input, a 2D convolutional layer needs a **4D tensor** with shape: **(batch, rows, cols, channels)**. Therefore, we need to precise the "channels" axis, which can be seen as the number of level of color of each input: 3 channels in our case. We will fix the dimension of images according to the VGG-16 network: (224, 224).


In [3]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D
import tensorflow.keras
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import glob
#!pip install tqdm
from tqdm import tqdm
from tensorflow.keras.layers import Flatten

### loading train data

Please read the code before running any of the cells!

In [4]:
import os
root = os.getcwd()
datasetRoot=root+'/raw-img/'
root

'/home/qlr/Programming/UCA_MSc-DS1/2_intro_DL/lecture_4'

In [5]:
# when processing time is long, it's nice to see the progress bar
from tqdm import tqdm

In [6]:
classes = ['mucca', 'elefante'] #, gatto', 'cavallo', 'scoiattolo', 'ragno', 'pecora', 'farfalla', 'gallina', 'cane']
nbClasses = len(classes)

#training data
rootTrain = datasetRoot+'train/'
print(rootTrain)

classLabel = 0
reducedSizePerClass = 1156+1492 #in order to reduce the number of images per class
totalImg = nbClasses * reducedSizePerClass
xTrain = np.empty(shape=(totalImg,224,224,3))
yTrain = []
first = True
i= 0
for cl in classes:
    listImages = glob.glob(rootTrain+cl+'/*')
    yTrain += [classLabel]*reducedSizePerClass #len(listImages) # note that here ...
    for pathImg in tqdm(listImages[:reducedSizePerClass]): # and here, we have reduced the data to be loaded (only 1000 per class)
        img = image.load_img(pathImg, target_size=(224,224))
        im = image.img_to_array(img)
        im = np.expand_dims(im, axis=0)
        im = preprocess_input(im)
        xTrain[i,:,:,:] = im
        i += 1
    classLabel += 1
print(len(yTrain))
print(xTrain.shape)

yTrain = tensorflow.keras.utils.to_categorical(yTrain, nbClasses)

  3%|▎         | 48/1492 [00:00<00:03, 477.92it/s]

/home/qlr/Programming/UCA_MSc-DS1/2_intro_DL/lecture_4/raw-img/train/


100%|██████████| 1156/1156 [00:03<00:00, 327.39it/s]

5296
(5296, 224, 224, 3)


In order to speed-up the time spent on this part of the lab, you may have noticed that we reduced the number of classes and the number of images per class. You can change these few lines of code if you want to work on the whole dataset.

### loading test data

In [7]:
#you will probably need to reduce the test dataset according to the train dataset rules
rootTest = datasetRoot+'test/'
classLabel = 0

totalTestImg = 0
for cl in classes:
    totalTestImg += len(glob.glob(rootTest+cl+'/*'))

xTest = np.empty(shape=(totalTestImg,224,224,3))
yTest = []
i = 0

for cl in classes:
    listImages = glob.glob(rootTest+cl+'/*')
    yTest += [classLabel]*len(listImages)
    for pathImg in listImages:
        img = image.load_img(pathImg, target_size=(224, 224))
        im = image.img_to_array(img)
        im = np.expand_dims(im, axis=0)
        im = preprocess_input(im)
        xTest[i,:,:,:] = im 
    classLabel += 1
print(len(yTest))
print(xTest.shape)
yTest = tensorflow.keras.utils.to_categorical(yTest, 2)

664
(664, 224, 224, 3)


In [8]:
yTrain[0]

array([1., 0.], dtype=float32)

# 1. Build your own CNN network

Start with the simplest CNN: 1 conv2D layer + 1 pooling + 1 dense layer.

In [9]:
model = Sequential()
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),padding='same'))
model.add(Conv2D(32,(3,3),padding='same',activation='relu'))
model.add(Flatten())
# you probably need to add something here
model.add(Dense(nbClasses, activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

Learn and test this network. 

In [10]:
history = model.fit(
    xTrain,
    yTrain,
    batch_size=8,
    epochs=3,
    validation_split=0.2,
    verbose=1
)

Epoch 1/3
530/530 [==============================] - 17s 28ms/step - loss: 0.5965 - accuracy: 0.9808 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/3
530/530 [==============================] - 14s 26ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/3
530/530 [==============================] - 14s 26ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [11]:
print(history.history)

{'loss': [0.1764330118894577, 0.0, 0.0], 'accuracy': [0.9962228536605835, 1.0, 1.0], 'val_loss': [0.0, 0.0, 0.0], 'val_accuracy': [1.0, 1.0, 1.0]}


In [12]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(xTest, yTest, batch_size=8)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(xTest[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
83/83 [==============================] - 1s 9ms/step - loss: 13.9967 - accuracy: 0.4383
test loss, test acc: [13.99670124053955, 0.4382530152797699]
Generate predictions for 3 samples
predictions shape: (3, 2)


You will encounter errors telling you that some dimensions are incompatible. Why ? How to correct ?

Try to modify the architecture and some of the parameters.

In [14]:
tensorflow.keras.backend.clear_session()

## About Dropout 

*Study this part only if you have time for it. It concerns the previous network but prefer to study first part II and come back here after.*


**Let's add dropout and activation functions to the network!**

In [16]:
from tensorflow.keras.layers import Dropout

model = Sequential(name='MLP model with dropout') 

model = Sequential()
model.add(Conv2D(256,(3,3),activation='relu',input_shape=(224,224,3)))
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(200,activation='relu'))
# adding dropout to the previous layer
model.add(Dropout(0.2))

model.add(Dense(nbClasses, activation='softmax'))

model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 256)     7168      
_________________________________________________________________
global_average_pooling2d (Gl (None, 256)               0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 200)               51400     
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 58,970
Trainable params: 58,970
Non-trainable params: 0
____________________________________________________

In [17]:
history = model.fit(
    xTrain,
    yTrain,
    batch_size=8,
    epochs=3,
    validation_split=0.2,
    verbose=1
)

Epoch 1/3
530/530 [==============================] - 20s 37ms/step - loss: 0.0963 - accuracy: 0.9848 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/3
530/530 [==============================] - 19s 35ms/step - loss: 1.6313e-09 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/3
530/530 [==============================] - 19s 35ms/step - loss: 1.8065e-10 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [18]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(xTest, yTest, batch_size=8)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(xTest[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
83/83 [==============================] - 1s 7ms/step - loss: 12.5191 - accuracy: 0.4383
test loss, test acc: [12.519134521484375, 0.4382530152797699]
Generate predictions for 3 samples
predictions shape: (3, 2)


# 2. Using a pre-learned network

## loading VGG-16 description part and adding layers to build our own classification network

In [ ]:
VGGmodel = VGG16(weights='imagenet', include_top=False)
features = VGGmodel.predict(xTrain)
print(features.shape)

# we will add layers to this feature extraction part of VGG network
m = VGGmodel.output
# we start with a global average pooling
m = GlobalAveragePooling2D()(m)
# and add a fully-connected layer
m = Dense(1024, activation='relu')(m)
# finally, the softmax layer for predictions (we have nbClasses classes)
predictions = Dense(nbClasses, activation='softmax')(m)

# global network
model = Model(inputs=VGGmodel.input, outputs=predictions)


Can you display the architecture of this entire network?

In [ ]:
# training
ourCallback = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=20, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
 
# training part I: training only the classification part (the end)
for layer in VGGmodel.layers:
    layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(xTrain, yTrain, epochs=2000, batch_size=128, validation_split=0.2, callbacks=[ourCallback],verbose=1)

# fine-tune the network

Fine-tune the entire network if you have enough computing ressouces, otherwise, carefully choose the layers you want to fine-tune.

In [ ]:
for i, layer in enumerate(VGGmodel.layers):
   print(i, layer.name)
model.summary()

In this example, we will fine-tune the last convolution block starting at layer number 15 (block5_conv).

In [ ]:
from tensorflow.keras.optimizers import RMSprop
for layer in model.layers[:15]:
   layer.trainable = False
for layer in model.layers[15:]:
   layer.trainable = True
#need to recompile the network
model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy',metrics=['accuracy'])
#and train again ...
model.fit(xTrain, yTrain, epochs=2000, batch_size=128, validation_split=0.2, callbacks=[ourCallback],verbose=1)

You already know how to evaluate the performances on the test dataset and display the confusion matrix. You can also modify the code that loads the test dataset in order to reduce it's size. Let's do it!

In [ ]:
#enter here your code for evaluation of performances

You are now free to experiments changes in the network:
* add a dense layer
* modify the number of neurons in dense layer(s)
* change the global average polling
* experiment other optimizers (SGD, Adam, ...)


...